In [ ]:
!pip install transformers==4.49.0

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification


In [ ]:
df = pd.read_csv("spam.csv", encoding='latin-1', low_memory=False, dtype=str)
df = df[['v1', 'v2']].copy()
df['label'] = df['v1'].str.strip().str.lower()
df['message'] = df['v2']
df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})
df = df.dropna(subset=['label_num', 'message'])
df['label_num'] = df['label_num'].astype(int)

texts = df['message'].astype(str).tolist()
labels = df['label_num'].values

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

In [ ]:
MODEL_NAME = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
train_encodings = tokenizer(
    train_texts,
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="tf"
)
val_encodings = tokenizer(
    val_texts,
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="tf"
)

train_labels_tf = tf.convert_to_tensor(train_labels)
val_labels_tf = tf.convert_to_tensor(val_labels)


In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2
)

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy("accuracy")

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
model.save_pretrained("spam_transformer_model")
tokenizer.save_pretrained("spam_transformer_model")

('spam_transformer_model/tokenizer_config.json',
 'spam_transformer_model/special_tokens_map.json',
 'spam_transformer_model/vocab.txt',
 'spam_transformer_model/added_tokens.json',
 'spam_transformer_model/tokenizer.json')

In [ ]:
model.save("spam_transformer_model_keras")

In [ ]:
import shutil

# Zip the folder
shutil.make_archive("spam_transformer_model", 'zip', "spam_transformer_model")

# Download it
from google.colab import files
files.download("spam_transformer_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [14]:
!cp spam_transformer_model.zip /content/drive/MyDrive/
